# Final Capstone Projects

Please refer to the [**Final Capstone Projects**](http://nbviewer.jupyter.org/github/jmportilla/Complete-Python-Bootcamp/tree/master/Final%20Capstone%20Projects/) folder to get all the info on final capstone project ideas and possible solutions!

# <center>Sort Excel/CSV File Utility </center>

<h2>Modules Import</h2>

In [1]:
#import openpyxl
#import csv
#import os

import pandas as pd
import tkinter as tk

from canvas import*
from tkinter import Tk     # from tkinter import Tk for Python 3.x
from tkinter.filedialog import askopenfilename
from tkinter.messagebox import askquestion
from tkinter.messagebox import askokcancel
from tkinter.messagebox import showerror
from tkinter import ttk
import getpass

#from tkinter import *
from functools import partial

#Constants Variables
supported_formats = ["xlsx","xls","xlsm","xlsb","csv"]

 <h2>Functions</h2>

In [2]:
def file_name_spliter(f_full_path):
    """Split the file name and his extension
    
    Parameters
    --------------------------------------
    f_full_path: the full path to the file
    """
    f_file_name = f_full_path.split("/")[-1].split(".")[0]
    f_file_extension =  f_full_path.split("/")[-1].split(".")[-1] 
    
    return f_file_name, f_file_extension

In [3]:
def right_format(f_file_name,f_file_extension):
    """Split the file name and his extension
    
    Parameters
    --------------------------------------
    f_file_name: the name of the file
    f_file_extension: the exension of the file
    """
    if f_file_extension not in supported_formats:
        msg = f"The file \"{f_file_name}\" is on \"{f_file_extension}\" format and is not supported by this program \n" \
              +"\nYou can see below the list of the supported format:\n\n" \
              + "\n".join(supported_formats)
        showerror(title='Error of file format', message=msg)
        return False
    else:
        return True

In [4]:
def get_data(f_path,f_extension):
    """Get Data from xlsx or csv File
    
    Parameters
    --------------------------------------
    f_path: the full path to the file
    f_extension: the extension of the file
    """
    if f_extension == "csv":
        data = pd.read_csv(f_path,delimiter=';')
    else:
        data = pd.read_excel(f_path)
    return data

<h2>Classes</h2>

In [24]:
class Application(tk.Tk):
    
    def __init__(self):
        tk.Tk.__init__(self)
        self.geometry("700x500")
        self.data = pd.DataFrame()
        self.create_widgets()
        
    def _quit(self):
        self.quit()
        self.destroy()
        
    def create_widgets(self):
        
        #creating elements
        self.resizable(False,False)
        self.frameData = tk.LabelFrame(self, relief='raised',text="Excel Data",  height=300, width=640)
        self.tv1 = ttk.Treeview(self.frameData)
        self.scrollY = tk.Scrollbar(self.tv1, orient=tk.VERTICAL, command=self.tv1.yview)
        self.scrollX = tk.Scrollbar(self.tv1, orient=tk.HORIZONTAL, command=self.tv1.xview)
        self.tv1.configure(yscrollcommand=self.scrollY.set, xscrollcommand=self.scrollX.set )
        
        self.frameCombine = tk.LabelFrame(self, relief='raised',height=100, width=310)
        self.frameHeaders = tk.LabelFrame(self.frameCombine, text="Choose the field whith you want to sort the file", height=100, width=310)
        self.labelTop1 = tk.Label(self.frameHeaders ,text = "Choose the field whith you want to sort the file")
        self.comboBox1 = ttk.Combobox(self.frameHeaders , values=[])
        
        self.frameOrder = tk.LabelFrame(self.frameCombine, text="Sorting order", height=100, width=310)
        self.comboBox2 = ttk.Combobox(self.frameOrder, values=["Ascending","Descending"])
        self.bouton = tk.Button(self.frameCombine, text="Sort the Data", command=self.sort_data,height=2)
        self.labelTop2 = tk.Label(self.frameOrder,text = "Sorting order")
        
        self.frameFile = tk.LabelFrame(self, relief='raised', text="File Management", height=50, width=640)
        self.bouton1 = tk.Button(self.frameFile, text="Load a file", command=self.choose_data,height=2)
        self.bouton2 = tk.Button(self.frameFile, text="Save the File", command="",height=2)
        self.bouton3 = tk.Button(self.frameFile, text="Quitter", command=self._quit,height=2)
        
        #placing all elements
        self.frameData.pack(side='top')
        self.tv1.place(relheight=1, relwidth=1) 
        self.scrollY.pack(side="right",fill="y")
        self.scrollX.pack(side="bottom",fill="x")
        
        self.frameCombine.pack(pady=10)
                               
        self.frameHeaders.pack(side="left",padx=10,pady=10)
        #self.labelTop1.pack(padx=5,pady=5)
        self.comboBox1.pack(padx=5,pady=5)
        
        self.frameOrder.pack(side="left",padx=10,pady=5)                     
        #self.labelTop2.pack(padx=5,pady=5)
        self.comboBox2.pack(padx=5,pady=5)
        self.comboBox2.current(1)
        self.bouton.pack(side="left",padx=10,pady=5) 
    
        self.frameFile.pack(padx=10,pady=5) 
        self.bouton1.pack(side="left",padx=10,pady=5)
        self.bouton2.pack(side="left",padx=10,pady=5)
        self.bouton3.pack(side="left",padx=10,pady=5)
   
    def load_data(self):
        
        for i in self.tv1.get_children():
            self.tv1.delete(i)
            
        #adding columns
        self.comboBox1['values']= list(self.data.columns)
        self.comboBox1.current(0)
        self.tv1["column"]= list(self.data.columns)
        self.tv1["show"] = "headings"

        for column in self.tv1["columns"]:
            self.tv1.heading(column, text=column) # let the column heading = column name

        #adding rows of data
        df_rows = self.data.to_numpy().tolist() # turns the dataframe into a list of lists
        for row in df_rows:
            self.tv1.insert("", "end", values=row)       
        
        
        
    #Choose a directory and a file
    def choose_data(self):
       
        #tk.Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
        file_path = askopenfilename() # show an "Open" dialog box and return the path to the selected file

        if file_path != "":
            
            #Get the extenion of the file
            file_name, file_extension = file_name_spliter (file_path)

            #Getting the data from the file
            if right_format(file_name, file_extension):
                    
                self.data = get_data(file_path,file_extension)
                
                self.load_data()
                  
               
                    
                #self.tv1["displaycolumns"]= "#all"
                
    #Sort he data by the selected COlumn
    def sort_data(self):

        if not self.data.empty:
            if self.comboBox2.get()=="Ascending":
                sort_type = True
            else:
                sort_type = False

            self.data.sort_values(by=[self.comboBox1.get()],ascending=sort_type,inplace=True)
            self.load_data()
            
    #Save the file
    

            
                

            
if __name__ == "__main__":
    app = Application()
    app.title("Sort Excel/CSV File Utility")
    app.mainloop()

In [6]:
tk.LabelFrame.winfo_width

<function tkinter.Misc.winfo_width(self)>

In [ ]:
#Choose the header to sort the data with 
#app =Tk.Tk()
#data.sort_values(by='First Name',inplace=True)
#data.to_excel("dataframe_to_xlsx.xlsx", index=False)
list(data.columns)

In [ ]:
app = Tk() 
app.geometry('500x500')

labelTop = tk.Label(app,text = "Choose the field whith you want to sort the file")
labelTop.grid(column=0, row=0)

labelHeaders = tk.Label(app,text = list(data.columns))
labelHeaders.grid(column=0, row=1)

comboExample = ttk.Combobox(app, values=list(data.columns))
comboExample.grid(column=0, row=2)
comboExample.current(0)

frame1 = tk.LabelFrame(app, text="Excel Data")
frame1.place(rely=0.65, relheight=1,relwidth=1)

app.mainloop()

In [ ]:
ws['A1'] = 42

In [ ]:
ws.rows

In [ ]:
# Rows can also be appended
ws.append([1, 2, 3])

# Python types will automatically be converted
import datetime
ws['A2'] = datetime.datetime.now()

# Save the file
wb.save("sample.xlsx")